In [ ]:
%pip install owslib shapely matplotlib ipyleaflet rasterio

### Imports

In [2]:
from owslib.wps import WebProcessingService, ComplexDataInput, Authentication
import json
import pandas as pd
import logging
import sys
import io
import time
from ipyleaflet import Map, GeoJSON, basemaps

In [3]:
# Set up owslib logging, to set relevant output
logger = logging.getLogger("owslib")
logger.setLevel(logging.WARNING)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.WARNING)
logger.addHandler(handler)

### Consts - Change these to affect the data retrieved

In [4]:
ENDPOINT="https://funcwps.ternlandscapes.org.au/wps/"
DATASET="aet"  # options are "smips" or "aet"
LAYER="ETa"  # options are "totalbucket" or "SMindex" for smips, or "ETa" for "aet".
ATDATE="2019-01-01"
# Enter your POLYGON as GeoJSON format.
MY_POLYGON='{"type": "Polygon", "coordinates": [[[151.39067721041408, -25.817716701946395], [153.37920260103908, -25.401622912943335], [154.82466059609754, -27.879438356653793], [152.27583247109754, -28.392903179661957], [150.18843012734754, -27.539015050742954], [150.23237543984754, -26.599878666580576], [151.39067721041408, -25.817716701946395]]]}'

### Define a connection to the server

In [5]:
# Enter your TERN API key as password
TERN_APIKEY = "N045_example_U0RySC5DDEF_replace_me_JOVpQVkdXezZMeC5ZWwlaNHxwDDQtDHsu"
AUTH = Authentication("apikey", password=TERN_APIKEY, shared=True)
server = WebProcessingService(ENDPOINT, auth=AUTH)

In [6]:
print('WPS Identification type: %s' % server.identification.type)
print('WPS Identification title: %s' % server.identification.title)
print('WPS Identification abstract: %s' % server.identification.abstract)
for operation in server.operations:
    print('WPS Operation: %s' % operation.name)
for process in server.processes:
    print('WPS Process: identifier=%s title=%s' % (process.identifier, process.title))

WPS Identification type: WPS
WPS Identification title: TERN Landscapes Functional Data WPS Server
WPS Identification abstract: The Functional Data Processing Server from TERN Landscapes
WPS Operation: GetCapabilities
WPS Operation: DescribeProcess
WPS Operation: Execute
WPS Process: identifier=sleep title=Sleep Process
WPS Process: identifier=temporalDrill title=Temporal Drill
WPS Process: identifier=subset1d title=SubSet 1D
WPS Process: identifier=subset2d title=SubSet 2D
WPS Process: identifier=subset3d title=SubSet 3D
WPS Process: identifier=stats2d title=Stats 2D
WPS Process: identifier=stats3d title=Stats 3D


In [7]:
process = server.describeprocess("stats2d")
print('WPS Process: identifier=%s' % process.identifier)
print('WPS Process: title=%s' % process.title)
print('WPS Process: abstract=%s' % process.abstract)
for input in process.dataInputs:
    print('Process input: identifier=%s, data type=%s, minOccurs=%d, maxOccurs=%d' % (input.identifier, input.dataType, input.minOccurs, input.maxOccurs))
for output in process.processOutputs:
    print('Process output: identifier=%s, data type=%s' % (output.identifier, output.dataType))

WPS Process: identifier=stats2d
WPS Process: title=Stats 2D
WPS Process: abstract=Return CSV file containing stats within given 2D polygon.
Process input: identifier=datasetId, data type=string, minOccurs=1, maxOccurs=1
Process input: identifier=atDate, data type=date, minOccurs=1, maxOccurs=1
Process input: identifier=returnType, data type=string, minOccurs=0, maxOccurs=1
Process input: identifier=polygon, data type=ComplexData, minOccurs=1, maxOccurs=1
Process output: identifier=download_link, data type=string
Process output: identifier=csv, data type=ComplexData


### Optional - Visualize shape

In [8]:
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=[-25.128, 135.588], zoom=4)
m.add_layer(GeoJSON(data=json.loads(MY_POLYGON)))
m

Map(center=[-25.128, 135.588], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

### Set up to send request, first create inputs

In [9]:
poly_input = ComplexDataInput(MY_POLYGON, mimeType="application/vnd.geo+json", schema="http://geojson.org/geojson-spec.html#polygon")

### Trigger process to begin (process executes asynchronously)

In [10]:
resp = server.execute(
    "stats2d",
    mode="async",
    inputs=[("datasetId", f"{DATASET}:{LAYER}"), ("polygon", poly_input), ("atDate", ATDATE)],
    output=[("csv", False, "text/csv")]  # Get stats as embedded file in result
)

### Wait for process to complete (polls every 4 seconds)

In [11]:
print("Waiting for process to complete...")
while True:
    resp.checkStatus(sleepSecs=0)
    print(f"Progress: {resp.percentCompleted}, {resp.statusMessage}")
    if resp.isComplete():
        break
    time.sleep(4)
print("Done")

Waiting for process to complete...
Progress: 0, Found 1 slices to slice up.
Progress: 0, Found 1 slices to slice up.
Progress: 0, Found 1 slices to slice up.
Progress: 100, Landscapes WPS Process succeeded
Done


### Extract CSV data and render it

In [13]:
first_output = next(iter(resp.processOutputs))
data = "".join(d for d in first_output.data)
df = pd.read_csv(io.StringIO(data))
df

,date,feature,min,max,mean,valid,stddev
0,2019-01-01T00:00:00+00:00,0,0.0,9617.0,3002.819645,47.471226,1232.98642
